# 0. 라이브러리 로드 및 기본 설정

In [1]:
import os
import librosa
import torch
from logger import utils
from tqdm import tqdm
from glob import glob
from pydub import AudioSegment
from logger.utils import traverse_dir

# Cuda setting
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Configure loading
args = utils.load_config('./configs/reflow.yaml')

# Set path
MP4_DATA_PATH   = 'preprocess/mp4'
ORIGINAL_PATH   = 'preprocess/original/'
DEMUCS_PATH     = 'preprocess/demucs/'
NORM_PATH       = 'preprocess/norm/'
TEMP_LOG_PATH   = 'temp_ffmpeg_log.txt' # 무음 감지 로그 임시 저장 위치

# 1. 데이터 준비
***
1. 전처리가 불필요한 경우 (모든 데이터가 배경음이 없고 특정 길이로 잘려 있는 경우),
    - 모든 데이터를 다음과 같이 data/train/audio 경로에 삽입하고
        ```
        # training dataset
        data/train/audio/aaa.wav
        data/train/audio/bbb.wav
        ...
        ```
    - 1-6. Validation 파일 분할 단계로 건너뛰기
***
2. 배경음은 제거되었지만 데이터가 너무 길어서 특정 길이로 잘라야 하는 경우,
    - 모든 데이터를 다음과 같이 preprocess/norm 경로에 삽입하고
        ```
        # training dataset
        preprocess/norm/aaa.wav
        preprocess/norm/bbb.wav
        ...
        ```
    - 1-4. Split 단계로 건너뛰기
***
3. 아무 처리도 되어 있지 않은 경우 (배경음도 제거해야 하고 데이터도 잘라야 하는 경우),
    - 모든 데이터를 다음과 같이 preprocess/original 경로에 삽입하고
        ```
        # training dataset
        preprocess/original/aaa.wav
        preprocess/original/bbb.wav
        ...
        ```
    - 1-2. Demucs 단계로 건너뛰기
***
4. 아무 처리도 되어 있지 않고 mp4 파일일 경우,
    - 모든 데이터를 다음과 같이 preprocess/mp4 경로에 삽입하고
        ```
        # training dataset
        preprocess/mp4/aaa.mp4
        preprocess/mp4/bbb.mp4
        ...
        ```
    - 1-1. mp4 데이터 변환 단계부터 차례대로 진행
***

## 1-1. mp4 데이터를 wav로 변환
- preprocess/mp4 경로에 있는 mp4 파일을 wav로 변환 후 preprocess/original 경로에 저장

In [ ]:
def mp4_to_wav(input_dir: str, input_file: str, output_dir: str):
    """ mp4 파일을 wav 형식으로 변환합니다.
    args:
        input_dir (str): 입력 mp4 파일의 경로
        input_file (str): 입력 mp4 파일의 이름
        output_dir (str): 출력 wav 파일의 경로
    """
    ext = os.path.splitext(input_file)[1][1:]

    if ext is not "mp4":
        return 
    else:
        track = AudioSegment.from_file(os.path.join(input_dir,input_file), format='mp4')
        track = track.set_frame_rate(44100)
        track.export(os.path.join(output_dir, input_file[:-4]+".wav"), format='wav')

filelist = traverse_dir(
    MP4_DATA_PATH,
    extension='mp4',
    is_pure=True,
    is_sort=True,
    is_ext=True)

for file in tqdm(filelist, desc="변환 중 "):
    mp4_to_wav(MP4_DATA_PATH, file, ORIGINAL_PATH)

## 1-2. Demucs (배경음 제거)
- preprocess/original 경로에 있는 wav 파일의 배경음(멜로디)을 제거 후 preprocess/demucs 경로에 저장

In [ ]:
from sep_wav import demucs

demucs(ORIGINAL_PATH, DEMUCS_PATH)

## 1-3. Normalize
- preprocess/demucs 경로에 있는 배경음이 제거된 데이터를 노멀라이즈 (sample rate 값 조정 가능) 후 preprocess/norm 경로에 저장

In [ ]:
from sep_wav import audio_norm

for filepath in tqdm(glob(DEMUCS_PATH+"*.wav"), desc="Normalizing... "):
    filename = os.path.splitext(os.path.basename(filepath))[0]
    out_filepath = os.path.join(NORM_PATH, filename) + ".wav"
    audio_norm(filepath, out_filepath, sample_rate = 44100)

## 1-4. Split
- preprocess/norm 경로에 있는 데이터를 15초 길이로 자른 후 data/train/audio 경로에 저장

In [ ]:
import subprocess

for filepath in tqdm(glob(NORM_PATH+"*.wav"), desc="Cutting... "):
    duration = librosa.get_duration(filename=filepath)
    max_last_seg_duration = 0
    sep_duration_final = 15
    sep_duration = 15

    while sep_duration > 4:
        last_seg_duration = duration % sep_duration
        if max_last_seg_duration < last_seg_duration:
            max_last_seg_duration = last_seg_duration
            sep_duration_final = sep_duration
        sep_duration -= 1

    filename = os.path.splitext(os.path.basename(filepath))[0]
    out_filepath = os.path.join(args.data.train_path,"audio", f"{filename}-%04d.wav")
    subprocess.run(f'ffmpeg -i "{filepath}" -f segment -segment_time {sep_duration_final} "{out_filepath}" -y', capture_output=True, shell=True)

## 1-5. 무음 제거
- data/train/audio 경로에 있는 음원들 중 무음(無音)인 파일을 찾아 제거

In [ ]:
from sep_wav import get_ffmpeg_args
import subprocess

for filepath in tqdm(glob(args.data.train_path+"/audio/*.wav"), desc="Removing... "):
    if os.path.exists(TEMP_LOG_PATH):
        os.remove(TEMP_LOG_PATH)

    ffmpeg_arg = get_ffmpeg_args(filepath)
    subprocess.run(ffmpeg_arg, capture_output=True, shell=True)

    start = None
    end = None

    with open(TEMP_LOG_PATH, "r", encoding="utf-8") as f:
        for line in f.readlines():
            line = line.strip()
            if "lavfi.silence_start" in line:
                start = float(line.split("=")[1])
            if "lavfi.silence_end" in line:
                end = float(line.split("=")[1])

    if start != None:
        if start == 0 and end == None:
            os.remove(filepath)
            
if os.path.exists(TEMP_LOG_PATH):
        os.remove(TEMP_LOG_PATH)

## 1-6. Validation 파일 분할
- data/train/audio 경로에 있는 데이터 중 일정한 비율 만큼을 나누어 data/val/audio 경로로 이동
    - 계산식: `max(SAMPLE_MIN, min(SAMPLE_MAX, int(len(all_files) * ratio)))`

In [ ]:
from types import SimpleNamespace
from draw import main

# Configure setting
configures = {
    # Optional : Default values are available
    'train'         :   os.path.abspath('.') + "/data/train/audio",
    'val'           :   os.path.abspath('.') + "/data/val/audio",
    'sample_rate'   :   1,
    'extensions'    :   ["wav", "flac"]
}
cmd = SimpleNamespace(**configures)

main(cmd)

# 2. 학습을 위한 데이터 전처리

In [3]:
from preprocess import preprocess
from ddsp.vocoder import F0_Extractor, Volume_Extractor, Units_Encoder
from diffusion.vocoder import Vocoder

# Get data
sample_rate = args.data.sampling_rate
hop_size = args.data.block_size
extensions = args.data.extensions

# Initialize f0 extractor
f0_extractor = F0_Extractor(
                    args.data.f0_extractor, 
                    args.data.sampling_rate, 
                    args.data.block_size, 
                    args.data.f0_min, 
                    args.data.f0_max)

# Initialize volume extractor
volume_extractor = Volume_Extractor(args.data.block_size)

# Initialize mel extractor
mel_extractor = None
use_pitch_aug = False
if args.model.type in ['Diffusion', 'DiffusionNew', 'DiffusionFast', 'RectifiedFlow']:
    mel_extractor = Vocoder(args.vocoder.type, args.vocoder.ckpt, device = device)
    if mel_extractor.vocoder_sample_rate != sample_rate or mel_extractor.vocoder_hop_size != hop_size:
        mel_extractor = None
        print("Unmatch vocoder parameters, mel extraction is ignored!")
    elif args.model.use_pitch_aug:
        use_pitch_aug = True

# Initialize units encoder
if args.data.encoder == 'cnhubertsoftfish':
        cnhubertsoft_gate = args.data.cnhubertsoft_gate
else:
    cnhubertsoft_gate = 10
units_encoder = Units_Encoder(
                    args.data.encoder, 
                    args.data.encoder_ckpt, 
                    args.data.encoder_sample_rate, 
                    args.data.encoder_hop_size,
                    cnhubertsoft_gate=cnhubertsoft_gate,
                    device = device)

# Preprocess training set
preprocess(args.data.train_path, f0_extractor, volume_extractor, mel_extractor, units_encoder, sample_rate, hop_size, device = device, use_pitch_aug = use_pitch_aug, extensions = extensions)

# Preprocess validation set
preprocess(args.data.valid_path, f0_extractor, volume_extractor, mel_extractor, units_encoder, sample_rate, hop_size, device = device, use_pitch_aug = use_pitch_aug, extensions = extensions)

 [Encoder Model] Content Vec
 [Loading] pretrain/contentvec/checkpoint_best_legacy_500.pt


2024-07-17 19:32:09 | INFO | fairseq.tasks.hubert_pretraining | current directory is c:\Users\PARK\Desktop\Project\DDSP-SVC
2024-07-17 19:32:09 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2024-07-17 19:32:09 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1, 'dropout_feature

Preprocess the audio clips in : data/train\audio


  0%|          | 0/92 [00:00<?, ?it/s]c:\Users\PARK\Desktop\Project\.venv\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
100%|██████████| 92/92 [00:21<00:00,  4.31it/s]


Preprocess the audio clips in : data/val\audio


100%|██████████| 2/2 [00:00<00:00,  4.44it/s]


# 3. 학습

In [ ]:
from train_reflow import training

training(args)

In [ ]:
""" # Old version : Use `combsub.yaml` as config
from train import training

training(args)
"""

# 4. 결과물 추출

In [ ]:
from types import SimpleNamespace
from main_reflow import inference

# Configure setting
configures = {
    # Values input Required
    'model_ckpt'        :   'exp/reflow-test/model_00000.pt',   # 모델 체크포인트 경로
    'input'             :   'data/train/audio/name_1234.wav',   # 원본 노래 파일 경로
    'output'            :   'output.wav',                       # 출력 파일 저장할 경로
    # Optional : Default values are available
    'device'            :   device,
    'spk_id'            :   '1',
    'spk_mix_dict'      :   'None',
    'key'               :   '0',
    'formant_shift_key' :   '0',
    'pitch_extractor'   :   'rmvpe',
    'f0_min'            :   '50',
    'f0_max'            :   '1100',
    'threhold'          :   '-60',
    'infer_step'        :   'auto',
    'method'            :   'auto',
    't_start'           :   'auto'
}
cmd = SimpleNamespace(**configures)

inference(cmd)

In [ ]:
""" # Old version : Use `combsub.yaml` as config
from types import SimpleNamespace
from main import inference

# Configure setting
configures = {
    # Values input Required
    'model_path'            :   'exp/combsub-test/model_00000.pt',  # 모델 체크포인트 경로
    'input'                 :   'data/train/audio/name_1234.wav',   # 원본 노래 파일 경로
    'output'                :   'output.wav',                       # 출력 파일 저장할 경로
    # Optional : Default values are available
    'device'                :   device,
    'spk_id'                :   '1',
    'spk_mix_dict'          :   'None',
    'key'                   :   '0',
    'enhance'               :   'true',
    'pitch_extractor'       :   'rmvpe',
    'f0_min'                :   '50',
    'f0_max'                :   '1100',
    'threhold'              :   '-60',
    'enhancer_adaptive_key' :   '0'
}
cmd = SimpleNamespace(**configures)

inference(cmd)
"""